<a href="https://colab.research.google.com/github/Suhail372/files_for_chatbot/blob/master/chatbot_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import torch
import numpy as np
import time
import random
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import streamlit as st

class VectorSearchWrapper:
    def __init__(self, location_is_hyd=False):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.EMBED_MODEL = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
        self.location_is_hyd = location_is_hyd

        self.hyd_json_file_path = 'combined files/cleaned_and_combined_hyd.json'
        self.blore_json_file_path = 'combined files/cleaned_and_combined_blore.json'

        self.model = SentenceTransformer(self.EMBED_MODEL, device=self.device)
        self.saved_vectors_path = 'saved_vectors'
        self.index_hyd = None
        self.index_blore = None
        self.embeddings_hyd = []
        self.embeddings_blore = []
        self.id_to_entry = {}
        self.run()

    def embedding(self, text_data):
        embedding = self.model.encode(text_data, convert_to_tensor=True, device=self.device)
        normalized_embedding = torch.nn.functional.normalize(embedding, p=2, dim=-1)
        return normalized_embedding.cpu().numpy()

    def preprocess_and_embed(self, json_file_path):
        embedded_list = []
        with open(json_file_path, 'r') as file:
            json_data = json.load(file)

        for entry in json_data:
            address = entry['Location']
            terms = [term.strip() for term in address.split(',')]
            replacable = ', '.join(terms[-4:]) if len(terms) > 4 else address

            entry['text data'] = entry['text data'].replace(address, replacable)
            text_data = entry["text data"].replace(f'Name: {entry["Name"]}', '')
            entry_id = entry.get("Id", None)

            if entry_id is not None:
                embedding = self.embedding(text_data)
                embedded_list.append({
                    "embedding": embedding,
                    "text": text_data,
                    "id": entry_id
                })

        return embedded_list

    def save_embeddings(self, embeddings, location_name):
        if not os.path.exists(self.saved_vectors_path):
            os.makedirs(self.saved_vectors_path)

        embeddings_array = np.vstack([entry["embedding"] for entry in embeddings])
        ids = [entry["id"] for entry in embeddings]
        texts = [entry["text"] for entry in embeddings]

        np.save(os.path.join(self.saved_vectors_path, f'embeddings_{location_name}.npy'), embeddings_array)
        with open(os.path.join(self.saved_vectors_path, f'metadata_{location_name}.json'), 'w') as f:
            json.dump({"ids": ids, "texts": texts}, f)

    def load_embeddings(self, location_name):
        embeddings_path = os.path.join(self.saved_vectors_path, f'embeddings_{location_name}.npy')
        metadata_path = os.path.join(self.saved_vectors_path, f'metadata_{location_name}.json')

        if os.path.exists(embeddings_path) and os.path.exists(metadata_path):
            embeddings_array = np.load(embeddings_path)
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)

            embeddings = [{"embedding": emb, "text": text, "id": id} for emb, text, id in zip(embeddings_array, metadata["texts"], metadata["ids"])]
            return embeddings
        return None

    def create_faiss_index(self):
        dimension = 384
        return faiss.IndexFlatL2(dimension)

    def insert_data(self, index, embeddings):
        embeddings_array = np.vstack([entry["embedding"] for entry in embeddings])
        index.add(embeddings_array)

    def run(self):
        self.embeddings_hyd = self.load_embeddings('hyd')
        self.embeddings_blore = self.load_embeddings('blore')

        if self.embeddings_hyd is None:
            print("Hyderabad embeddings not found. Preprocessing and creating new embeddings.")
            self.embeddings_hyd = self.preprocess_and_embed(self.hyd_json_file_path)
            self.save_embeddings(self.embeddings_hyd, 'hyd')
        if self.embeddings_blore is None:
            print("Bangalore embeddings not found. Preprocessing and creating new embeddings.")
            self.embeddings_blore = self.preprocess_and_embed(self.blore_json_file_path)
            self.save_embeddings(self.embeddings_blore, 'blore')

        self.index_hyd = self.create_faiss_index()
        self.index_blore = self.create_faiss_index()

        self.insert_data(self.index_hyd, self.embeddings_hyd)
        self.insert_data(self.index_blore, self.embeddings_blore)

    def search_faiss(self, query, k=3):
        query_embedding = self.embedding(query).reshape(1, -1)
        index = self.index_hyd if self.location_is_hyd else self.index_blore
        embeddings = self.embeddings_hyd if self.location_is_hyd else self.embeddings_blore
        json_file_path = self.hyd_json_file_path if self.location_is_hyd else self.blore_json_file_path

        distances, indices = index.search(query_embedding, k)

        results = [{"id": embeddings[idx]["id"], "text": embeddings[idx]["text"]} for idx in indices[0]]
        with open(json_file_path, 'r') as file:
            data = json.load(file)

        for i in results:
            for j in data:
                if i['id'] == j['Id']:
                    i['text'] = j['text data']

        return results

class LLMHandler:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.llama_tokenizer, self.llama_model = self.load_llama_model()

    def load_llama_model(self):
        llama_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
        llama_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct").to(self.device)
        llama_model.temperature = 1.0
        llama_model.top_p = 0.95
        return llama_tokenizer, llama_model

    def generate_query(self, query, history):
        input_prompt = f"""
        <>
            Generate a question similar to previous queries, and given user query, make it concise and do not generate an answer to the query.
        <>
        [INST]
        History: {history}
        Query: {query}
        [/INST]
        """
        inputs = self.llama_tokenizer(input_prompt, return_tensors="pt", padding=True).to(self.device)
        outputs = self.llama_model.generate(**inputs, max_new_tokens=1000, pad_token_id=self.llama_model.config.eos_token_id)
        result_prompt = self.llama_tokenizer.batch_decode(outputs)[0]
        return result_prompt.replace(input_prompt, "").strip()

    def is_query(self, query):
        query = query.lower()
        my_list = ["school", "schools", "facilities", "amenities", "sports", "faculty", "fees", "institute", "organisation", "org", "inst", "scl", "schol"]
        return any(item in query for item in my_list)

    def requires_context(self, query):
        query = query.lower()
        context_keywords = ["previous", "last", "before", "history", "context", "same"]
        return any(item in query for item in context_keywords)

    def generate_chat_response(self, query, data):
        input_prompt = f"""
        <>You are a chatbot assistant that recommends schools to the user by describing the school's information. Generate an answer for the query given using the search results provided and nothing else in the response field.
        <>
        [INST]
        User Query: {query}
        Search results: {data}
        [/INST]
        """
        inputs = self.llama_tokenizer([input_prompt], return_tensors="pt").to(self.device)
        outputs = self.llama_model.generate(**inputs, max_new_tokens=1000, pad_token_id=self.llama_model.config.eos_token_id)
        result_prompt = self.llama_tokenizer.decode(outputs[0])
        return result_prompt.replace(input_prompt, "").strip()

    def get_not_school_related_response(self):
        not_school_related_responses = [
            "I'm sorry, but I can only provide information related to schools. How can I assist you with a school-related query today?",
            "It looks like your question isn't related to schools. Could you please ask something about schools so I can help you better?",
            "I'm here to help with school-related questions! If you have any queries about schools, feel free to ask.",
            "Oops! I can only assist with questions about schools. Please let me know if there's anything you need to know about schools.",
            "I'm not equipped to handle this query. Do you have any school-related queries?",
            "I'm focused on answering school-related questions. Can you ask something about schools?"
        ]
        return random.choice(not_school_related_responses)

class Chatbot(VectorSearchWrapper, LLMHandler):
    def __init__(self, location_is_hyd=False):
        VectorSearchWrapper.__init__(self, location_is_hyd)
        LLMHandler.__init__(self)

    def process_query(self, user_query, user_history):
        if self.is_query(user_query):
            if self.requires_context(user_query):
                user_query = self.generate_query(user_query, user_history)
            search_results = self.search_faiss(user_query, k=3)
            formatted_results = "\n".join([f"{result['text']}" for result in search_results])
            chat_response = self.generate_chat_response(user_query, formatted_results)
            return chat_response
        else:
            return self.get_not_school_related_response()

# Streamlit app
st.title("School Information Chatbot")

st.sidebar.title("Choices sidebar")
choice = st.sidebar.radio("Choose location:", ("Hyderabad", "Bangalore"))
st.sidebar.write(f"Current location: {choice}")
location_is_hyd = True if choice == "Hyderabad" else False

# Initialize the chatbot once and store it in session state
if "chatbot" not in st.session_state or st.session_state.chatbot.location_is_hyd != location_is_hyd:
    st.session_state.chatbot = Chatbot(location_is_hyd)

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

prompt = st.chat_input("Enter your prompt")
if prompt:
    with st.chat_message("user"):
        st.markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    chatbot = st.session_state.chatbot
    user_history = st.session_state.get("history", "")
    response = chatbot.process_query(prompt, user_history)
    st.session_state.history = user_history + f"\n{prompt}"

    with st.spinner("Generating response..."):
        time.sleep(2)  # Simulate longer processing time
        with st.chat_message("assistant"):
            st.markdown(response)
    st.session_state.messages.append({"role": "assistant", "content": response})
